In [11]:
import pylab
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from scipy.spatial import distance
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from mpl_toolkits.axes_grid1 import make_axes_locatable
import string

# Network structure
Variables that define the structure of the network model in the following terms:
- dimensions of the encoded space, 
- number of neurons, 
- receptive fields (RF) centres.

In [12]:
## Tactile area
# Tactile area dimensions (cm)
xt_dim = 10
yt_dim = 5

# Tactile neurons (count)
Mt = 20
Nt = 10

# Tactile RF centres (cm)
xt = np.arange(1, Mt + 1) * 0.5
yt = np.arange(1, Nt + 1) * 0.5

## Auditory area
# Auditory area dimensions (cm)
xa_dim = 200
ya_dim = 30

# Auditory neurons (count)
Ma = 20
Na = 3

# Auditory RF centres (cm)
xa = (np.arange(1, Ma + 1) * 10) - 5
ya = (np.arange(1, Na + 1) * 10) - 15

# Network fixed parameters
Variables that define the values of the parameters that remain fixed in our study. These parameters are related to: 
- the RF of the unisensory neurons,
- the activation functions of both unisensory and multisensory neurons, 
- the visual and tactile stimuli that is administered during the experiment simulation.  

In [13]:
### Model fixed parameters

## Unisensory receptive fields
phit_0 = 1
sigmat_phi = 0.5
phia_0 = 1
sigmaa_phi = 10

## Neuronal activity

# Tactile neurons
ft_min = -0.12
ft_max = 1
qt_c = 19.43
rt = 0.34

# Auditory neurons
fa_min = -0.12
fa_max = 1
qa_c = 19.43
ra = 0.34

# Multisensory neuron
fm_min = 0
fm_max = 1
qm_c = 12
rm = 0.6

tau = 20

## External stimuli
# Tactile stimuli
it_0 = 2.5
sigmat_i = 0.3
xt_0 = 5  # cm
yt_0 = 2.5  # cm

# Auditory stimuli
ia_0 = 3.6
sigmaa_i = 0.3
ya_0 = 5  # cm
xa_0 = 100  # cm

# External stimuli functions

In [14]:
## External stimulus
def stimt(x, y):
    """Generate tactile stimuli for the network for a given set of spatial coordinates.

    Args:
        x (number): Spatial coordinate in the x-axis.
        y (number): Spatial coordinate in the y-axis.

    Returns:
        I (2D np.array): Tactile stimuli for the tactile area.
    """

    # Compute tactile stimulus
    I = (it_0) * np.exp(
        -(np.square(xt_0 - x) + np.square(yt_0 - y)) / (2 * np.square(sigmat_i))
    )

    return I


def stima(x, y, xa_0):
    """Generate auditory stimuli for the network.

    Args:
        x (number): Spatial coordinate in the x-axis.
        y (number): Spatial coordinate in the y-axis.
        xa_0 (number): Central point of the auditory stimulus in the x-axis (cm).

    Returns:
        I (2D np.array): Auditory stimuli for the auditory area.
    """
    # Compute auditory stimulus
    I = (ia_0) * np.exp(
        -(np.square(xa_0 - x) + np.square(ya_0 - y)) / (2 * np.square(sigmaa_i))
    )

    return I

# Receptive fields functions

In [15]:
def phit(x, y):
    """Obtain the RF of the tactile neurons for a given set of spatial coordinates.

    Args:
        x (number): Spatial coordinate in the x-axis (cm).
        y (number): Spatial coordinate in the y-axis (cm).

    Returns:
        phi (2D np.array): RF of the tactile area for the given spatial coordinates.
    """

    # Build the RF matrix
    phi = np.zeros((Mt, Nt))

    # Compute the RF for the given spatial coordinates
    for i in range(Mt):
        for j in range(Nt):
            phi[i][j] = phit_0 * np.exp(
                -(
                    (np.square(xt[i] - x) + np.square(yt[j] - y))
                    / (2 * np.square(sigmat_phi))
                )
            )
    return phi


def phia(x, y):
    """Obtain the RF of the auditory neurons for a given set of spatial coordinates.

    Args:
        x (number): Spatial coordinate in the x-axis (cm).
        y (number): Spatial coordinate in the y-axis (cm).

    Returns:
        phi (2D np.array): RF of the auditory area for the given spatial coordinates.
    """

    # Build the RF matrix
    phi = np.zeros((Ma, Na))

    # Compute the RF for the given spatial coordinates
    for i in range(Ma):
        for j in range(Na):
            phi[i][j] = phia_0 * np.exp(
                -(
                    (np.square(xa[i] - x) + np.square(ya[j] - y))
                    / (2 * np.square(sigmaa_phi))
                )
            )
    return phi


def phi_computation():
    """Compute the RF of the unisensory areas for the discreteised spatial coordinates
        in the auditory and tactile spaces.

    Returns:
        phi_t (4D np.array): RF of the tactile area for the discretised spatial coordinates in the tactile space.
        phi_a (4D np.array): RF of the auditory area for the discretised spatial coordinates in the auditory space.
    """

    # Define the step size (cm) for the integral computation
    dif = 0.2

    # Calculate the discretised spatial coordinates
    xt_i = np.arange(0, xt_dim + dif, dif)
    yt_n = np.arange(0, yt_dim + dif, dif)
    xa_i = np.arange(0, xa_dim + dif, dif)
    ya_n = np.arange(0, ya_dim + dif, dif)

    # Build the RF matrices
    phi_t = np.zeros((Mt, Nt, len(xt_i), len(yt_n)))
    phi_a = np.zeros((Ma, Na, len(xa_i), len(ya_n)))

    # Compute the RF for the discretised spatial coordinates
    for k in range(len(xt_i)):
        for l in range(len(yt_n)):
            phi_t[:, :, k, l] = phit(xt_i[k], yt_n[l])

    for k in range(len(xa_i)):
        for l in range(len(ya_n)):
            phi_a[:, :, k, l] = phia(xa_i[k], ya_n[l])

    return phi_t, phi_a


### Receptive fields computation
"""Computation of the RF of the unisensory areas for the discreteised spatial coordinates 
        in the auditory and tactile spaces using a step size of 0.2 cm. We compute this outside 
        the stimulus functions PHIv and PHIt to reduce the running time of the model. 
"""
phi_t, phi_a = phi_computation()

# Synapses functions

In [16]:
def Lw(Lt_ex, Lt_in, sigmat_ex, sigmat_in, La_ex, La_in, sigmaa_ex, sigmaa_in):
    """Obtain the recurrent synaptic weights of the unisensory neurons.

    Args:
        Lt_ex (number): Amplitude of excitatory synapses in the tactile area.
        Lt_in (number): Amplitude of inhibitory synapses in the tactile area.
        sigmat_ex (number): Extension of excitatory synapses (cm) in the tactile area.
        sigmat_in (number): Extension of inhibitory synapses (cm) in the tactile area.

        La_ex (number): Amplitude of excitatory synapses in the auditory area.
        La_in (number): Amplitude of inhibitory synapses in the auditory area.
        sigmaa_ex (number): Extension of excitatory synapses (cm) in the auditory area.
        sigmaa_in (number): Extension of inhibitory synapses (cm) in the auditory area.

    Returns:
        Lt (2D np.array): Recurrent synaptic weights of the tactile area.
        La (2D np.array): Recurrent synaptic weights of the auditory area.
    """

    # Build the recurrent connectivity matrix of the tactile area.
    Lt = np.zeros((Mt * Nt, Mt * Nt))

    # Compute the recurrent connectivity matrix of the tactile area.
    for i in range(Mt * Nt):
        for j in range(Mt * Nt):
            if i == j:
                Lt[i, j] = 0
            else:
                Dtx = xt[np.floor_divide(i, Nt)] - xt[np.floor_divide(j, Nt)]
                Dty = yt[np.remainder(i, Nt)] - yt[np.remainder(j, Nt)]
                Lt[i, j] = Lt_ex * np.exp(
                    -(np.square(Dtx) + np.square(Dty)) / (2 * np.square(sigmat_ex))
                ) - Lt_in * np.exp(
                    -(np.square(Dtx) + np.square(Dty)) / (2 * np.square(sigmat_in))
                )

    # Build the recurrent connectivity matrix of the auditory area.
    La = np.zeros((Ma * Na, Ma * Na))

    # Compute the recurrent connectivity matrix of the auditory area.
    for i in range(Ma * Na):
        for j in range(Ma * Na):
            if i == j:
                La[i, j] = 0
            else:
                Dax = xa[np.floor_divide(i, Na)] - xa[np.floor_divide(j, Na)]
                Day = (
                    ya[np.remainder(i, Na)] - ya[np.remainder(j, Na)]
                )  # before was remainder-1
                La[i, j] = La_ex * np.exp(
                    -(np.square(Dax) + np.square(Day)) / (2 * np.square(sigmaa_ex))
                ) - La_in * np.exp(
                    -(np.square(Dax) + np.square(Day)) / (2 * np.square(sigmaa_in))
                )

    return Lt, La


def Fw(Wt_0, Wa_0, Bt_0, Ba_0, k1, k2, lim, alpha):
    """Feedforward and feedback synaptic weights of the unisensory neurons.

    Args:
        Wt_0 (number): Maximum value of the feedforward synapses in the tactile area.
        Wa_0 (number): Maximum value of the feedforward synapses in the auditory area.

        Bt_0 (number): Maximum value of the feedback synapses in the tactile area.
        Ba_0 (number): Maximum value of the feedback synapses in the auditory area.

        k1 (number): Fast decay rate.
        k2 (number): Slow decay rate.
        lim (number): Boundary of the space on and near the hand.
        alpha (number): Relative amplitude of each exponential.

    Returns:
        Wt (2D np.array): Feedforward synaptic weights in the tactile area.
        Wa (2D np.array): Feedforward synaptic weights in the auditory area.
        Bt (2D np.array): Feedback synaptic weights in the tactile area.
        Ba (2D np.array): Feedback synaptic weights in the auditory area.
    """

    # Build the feedforward and feedback synapses matrices
    Bt = np.ones((Mt, Nt)) * Bt_0
    Wt = np.ones((Mt, Nt)) * Wt_0
    Ba = np.zeros((Ma, Na))
    Wa = np.zeros((Ma, Na))

    # Compute the feedforward and feedback synapses in the auditory area
    for i in range(Ma):
        for j in range(Na):
            if xa[i] < lim:
                D = 0
            else:
                D = distance.euclidean((xa[i], ya[j]), (lim, ya[j]))
            Ba[i, j] = alpha * Ba_0 * np.exp(-D / k1) + (1 - alpha) * Ba_0 * np.exp(
                -D / k2
            )
            Wa[i, j] = alpha * Wa_0 * np.exp(-D / k1) + (1 - alpha) * Wa_0 * np.exp(
                -D / k2
            )

    return Wt, Wa, Bt, Ba

# Neural input functions

In [31]:
## Unisensory
def PHIt():
    """Compute the external input of the tactile area as the convolution
        between the external stimulus and the RF of the tactile area.

    Returns:
        PHI (2D np.array): External input of the tactile area.
    """

    # Define the step size (cm) for the integral computation
    dif = 0.2

    # Calculate the discretised spatial coordinates of the tactile space
    xt_i = np.arange(0, xt_dim + dif, dif)
    yt_n = np.arange(0, yt_dim + dif, dif)

    # Build the external input matrix
    PHI = np.zeros((Mt, Nt, len(xt_i), len(yt_n)))

    # Compute the external input using the histogram rule
    for k in range(len(xt_i)):
        for l in range(len(yt_n)):
            PHI[:, :, k, l] = np.multiply(phi_t[:, :, k, l], stimt(xt_i[k], yt_n[l]))
    PHI = np.sum(PHI, axis=3)
    PHI = np.sum(PHI, axis=2)

    return PHI


def PHIa(xa_0):
    """Compute the external input of the auditory area as the convolution
        between the external stimulus and the RF of the tactile area.

    Args:
        xa_0 (number): Central point of the auditory stimulus in the x-axis (cm).

    Returns:
        PHI (2D np.array): External input of the auditory area.
    """

    # Define the step size (cm) for the integral computation
    dif = 0.2

    # Calculate the discretised spatial coordinates of the visual space
    xa_i = np.arange(0, xa_dim + dif, dif)
    ya_n = np.arange(0, ya_dim + dif, dif)

    # Build the external input matrix
    PHI = np.zeros((Ma, Na, len(xa_i), len(ya_n)))

    # Compute the external visual input using the histogram rule
    for k in range(len(xa_i)):
        for l in range(len(ya_n)):
            PHI[:, :, k, l] = np.multiply(
                phi_a[:, :, k, l], stima(xa_i[k], ya_n[l], xa_0)
            )
    PHI = np.sum(PHI, axis=3)
    PHI = np.sum(PHI, axis=2)

    return PHI


## Lateral
def LIt(z, Lt):
    """Compute the lateral input of the tactile area as the product
        between neural activity and recurrent synaptic weights.

    Args:
        z (2D np.array): A matrix of shape (MtxNt) with the activity of tactile neurons.

    Returns:
        LI (2D np.array): Lateral input of the tactile area.
    """

    # Build the lateral input matrix
    LI = np.zeros(Mt * Nt)

    # Compute the lateral input
    z = np.reshape(z, (1, Mt * Nt))
    for i in range(Mt * Nt):
        LI[i] = np.sum(np.multiply(Lt[i, :], z[0, :]))
    LI = np.reshape(LI, (Mt, Nt))

    return LI


def LIa(z, La):
    """Compute the lateral input of the auditory area as the product
        between neural activity and recurrent synaptic weights.

    Args:
        z (2D np.array): A matrix of shape (MaxNa) with the activity of auditory neurons.

    Returns:
        LI (2D np.array): Lateral input of the auditory area.
    """

    # Build the lateral input matrix
    LI = np.zeros(Ma * Na)

    # Compute the lateral input
    z = np.reshape(z, (1, Ma * Na))
    for i in range(Ma * Na):
        LI[i] = np.sum(np.multiply(La[i, :], z[0, :]))
    LI = np.reshape(LI, (Ma, Na))

    return LI


## Feedback
def bt(z, Bt):
    """Compute the feeback input of the tactile area as the product
        between neural activity and feedback synaptic weights.

    Args:
        z (number): Activity of the multisensory neuron.
        Bt (2D np.array): Feedback synaptic weights in the tactile area.

    Returns:
        bt (2D np.array): Feedback input of the tactile area.
    """

    # Compute the feedback input
    bt = np.multiply(Bt, z)

    return bt


def ba(z, Ba):
    """Compute the feeback input of the auditory area as the product
        between neural activity and feedback synaptic weights.

    Args:
        z (number): Activity of the multisensory neuron.
        Ba (2D np.array): Feedback synaptic weights in the auditory area.

    Returns:
        ba (2D np.array): Feedback input of the auditory area.
    """

    # Compute the feedback input
    ba = np.multiply(Ba, z)

    return ba

# Neural activity functions

In [18]:
def psit(qt):
    """Compute the activity of tactile neurons from its overall input.

    Args:
        qt (2D np.array): State variable of tactile neurons after receiving input.

    Returns:
        psi (2D np.array): Neural activity of the tactile area.
    """

    # Build the activity matrix
    psi = np.zeros((Mt, Nt))

    # Compute the neural activity
    for i in range(Mt):
        for j in range(Nt):
            psi[i, j] = (ft_min + ft_max * np.exp((qt[i, j] - qt_c) * rt)) / (
                1 + np.exp((qt[i, j] - qt_c) * rt)
            )

    return psi


def psia(qa):
    """Compute the activity of auditory neurons from its overall input.

    Args:
        qa (2D np.array): State variable of auditory neurons after receiving input.

    Returns:
        psi (2D np.array): Neural activity of the auditory area.
    """

    # Build the activity matrix
    psi = np.zeros((Ma, Na))

    # Compute the neural activity
    for i in range(Ma):
        for j in range(Na):
            psi[i, j] = (fa_min + fa_max * np.exp((qa[i, j] - qa_c) * ra)) / (
                1 + np.exp((qa[i, j] - qa_c) * ra)
            )

    return psi


def psim(qm):
    """Compute the activity of the multisensory neuron from its overall input.

    Args:
        qm (number): State variable of the multisensory neuron after receiving input.

    Returns:
        psi (number): Neural activity of the multisensory area.
    """

    # Compute the neural activity
    psi = (fm_min + fm_max * np.exp((qm - qm_c) * rm)) / (1 + np.exp((qm - qm_c) * rm))

    return psi

# Audio-tactile experiment function

In [1]:
## Experiment function
def experimentrun(a_distances, time, k1, k2, lim, alpha):
    """Compute the RT of the network for the given distance points.

    Args:
        a_distances (1D array): Vector with the distances at which the auditory stimuli is delivered (cm).
        time (number): Running time of the experiment (ms)

        k1 (number): Fast decay rate.
        k2 (number): Slow decay rate.
        lim (number): Boundary of the space on and near the hand.
        alpha (number): Relative amplitude of each exponential.

    Returns:
        RTs (1D np.array): Vector with the RTs at the given distance points (ms).
        ZMs (2D np.array): Matrix with the activity of the multisensory neuron during the experiment at each distance point.
        ZTs (3D np.array): Array with the final activity of the tactile neurons at each distance point.
        ZAs (3D np.array): Array with the final activity of the auditory neurons at each distance point.
    """

    # Setup network synapses
    Lt, La = Lw(0.15, 0.05, 1, 4, 0.15, 0.05, 20, 80)
    Wt, Wa, Bt, Ba = Fw(6.5, 6.5, 2.5, 2.5, k1, k2, lim, alpha)

    # Network computing parameters
    dt = 0.4
    dtau = dt / tau

    # Experiment parameters
    timesteps = int(time / dt)
    ndist = len(a_distances)

    # Build matrices to register network activity during the experiment
    RTs = np.zeros(ndist)
    ZTs = np.zeros((Mt, Nt, ndist))
    ZAs = np.zeros((Ma, Na, ndist))

    qt = np.zeros((Mt, Nt, timesteps + 1, ndist))
    ut = np.zeros((Mt, Nt, timesteps + 1, ndist))
    zt = np.zeros((Mt, Nt, timesteps + 1, ndist))
    pt = np.zeros((Mt, Nt, timesteps + 1, ndist))

    qa = np.zeros((Ma, Na, timesteps + 1, ndist))
    ua = np.zeros((Ma, Na, timesteps + 1, ndist))
    za = np.zeros((Ma, Na, timesteps + 1, ndist))
    pa = np.zeros((Ma, Na, timesteps + 1, ndist))

    qm = np.zeros((timesteps + 1, ndist))
    um = np.zeros((timesteps + 1, ndist))
    zm = np.zeros((timesteps + 1, ndist))
    pm = np.zeros((timesteps + 1, ndist))

    rt = np.zeros((timesteps + 1, ndist))

    # Generate a tactile stimulus
    ti = PHIt()

    # Run the experiment
    for d in range(ndist):
        xa_0 = a_distances[d]  # How far the sound is presented.
        ai = PHIa(xa_0)  # Generates an auditory input

        for i in range(timesteps):
            # Tactile activity
            ut[:, :, i + 1, d] = ti + LIt(zt[:, :, i, d], Lt) + bt(zm[i, d], Bt)
            qt[:, :, i + 1, d] = qt[:, :, i, d] + dtau * (
                -qt[:, :, i, d] + ut[:, :, i, d]
            )
            pt[:, :, i + 1, d] = psit(qt[:, :, i, d])
            zt[:, :, i + 1, d] = pt[:, :, i, d] * np.heaviside(pt[:, :, i, d], 0)

            # Auditory activity
            ua[:, :, i + 1, d] = ai + LIa(za[:, :, i, d], La) + ba(zm[i, d], Ba)
            qa[:, :, i + 1, d] = qa[:, :, i, d] + dtau * (
                -qa[:, :, i, d] + ua[:, :, i, d]
            )
            pa[:, :, i + 1, d] = psia(qa[:, :, i, d])
            za[:, :, i + 1, d] = pa[:, :, i, d] * np.heaviside(pa[:, :, i, d], 0)

            # Multisensory activity
            um[i + 1, d] = np.sum(np.multiply(Wt, zt[:, :, i, d])) + np.sum(
                np.multiply(Wa, za[:, :, i, d])
            )
            qm[i + 1, d] = qm[i, d] + dtau * (-qm[i, d] + um[i, d])
            pm[i + 1, d] = psim(qm[i, d])
            zm[i + 1, d] = pm[i, d] * np.heaviside(pm[i, d], 0)

            # Network response
            rt[i + 1, d] = np.any(zt[:, :, i, d] > (0.9 * ft_max))

        # Compute RT
        RTs[d] = np.argmax(rt[:, d]) * dt

        # Save relevant network activity
        ZMs = zm
        ZTs[:, :, :, d] = zt[:, :, timesteps, d]
        ZAs[:, :, d] = za[:, :, timesteps, d]

    return RTs, ZMs, ZTs, ZAs

# Sigmoid fitting functions 

In [30]:
def RTsig(data, cp, h):
    """Compute sigmoidal RT for the given distance points.

    Args:
        data (1D np.array): Distance points (cm).
        cp (number): Central point of the sigmoid.
        h (number): Slope parameter of the sigmoid (slope = 1/h).

    Returns:
        sig (1D np.array): Vector with RT for the given distance points.
    """

    # Obtains the upper and lower bounds from global environment
    global ymin
    global ymax

    # Compute the sigmoid
    sig = (ymin + ymax * np.exp((data - cp) / h)) / (1 + np.exp((data - cp) / h))

    return sig


## Sigmoid function fitting
def sigfit(x, y):
    """Fit the RT data to a sigmoidal function.

    Args:
        x (1D np.array): Distance points (cm).
        y (1D np.array) : RT data to be fitted.

    Returns:
        dc (number): Central point of the sigmoid.
        h (number): Slope parameter of the sigmoid (slope = 1/h).
    """

    # Obtains the upper and lower bounds from global environment
    global ymin
    global ymax

    # Defines starting points and boundaries for the fitting
    k_0 = (ymax - ymin) / (x[-1] - x[0])
    initial_slope = (ymax - ymin) / (4 * k_0)
    middle_x = np.max(x) / 2
    init_guess = [middle_x, initial_slope]
    boundaries = ([0, float("-inf")], [float("inf"), float("inf")])

    # Fits the data
    popt, pcov = curve_fit(
        RTsig,
        x,
        y,
        p0=init_guess,
        method="trf",
        ftol=1e-8,
        xtol=1e-8,
        maxfev=10000,
        bounds=boundaries,
    )
    sigpar = np.asarray(popt)
    dc = sigpar[0]
    h = sigpar[1]

    return dc, h